# Choose Dataset

In [147]:
using Revise,NeuroAnalysis,Interact
droot = "../VLabDataExport";rroot = "../Result/LZ-Alex";test="Laser"
sdir = dropdown(filter!(x->x[1]!='.',readdir(droot)),label="Subject")
fname = map(x->dropdown(getvlabfile(subject=x,test=test,dir="$droot/$x"),label="Experiment"),signal(sdir))
display.([sdir,fname]);

Interact.Options{:Dropdown,String}(89: "input-41" = C1701 String , "Subject", "C1701", "C1701", 1, Interact.OptionDict(DataStructures.OrderedDict("C1701"=>"C1701","C1702"=>"C1702","C1801"=>"C1801","LSR1801"=>"LSR1801","LZR1602"=>"LZR1602","LZR1603"=>"LZR1603","LZR1604"=>"LZR1604","LZR1801"=>"LZR1801","LZR1802"=>"LZR1802","metadata.mat"=>"metadata.mat"…), Dict("LZR1602"=>"LZR1602","LZR1802"=>"LZR1802","metadata.mat"=>"metadata.mat","C1702"=>"C1702","LZR1604"=>"LZR1604","C1701"=>"C1701","C1801"=>"C1801","LZR1603"=>"LZR1603","LZR1801"=>"LZR1801","LSR1801"=>"LSR1801"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:Dropdown,String}(91: "input-42" = C1701_V2Site2_3000Ch18_Laser_1.mat String , "Experiment", "C1701_V2Site2_3000Ch18_Laser_1.mat", "C1701_V2Site2_3000Ch18_Laser_1.mat", 1, Interact.OptionDict(DataStructures.OrderedDict("C1701_V2Site2_3000Ch18_Laser_1.mat"=>"C1701_V2Site2_3000Ch18_Laser_1.mat","C1701_V2Site2_3000Ch21_Laser_1.mat"=>"C1701_V2Site2_3000Ch21_Laser_1.mat","C1701_V2Site2_3000Ch22_Laser_1.mat"=>"C1701_V2Site2_3000Ch22_Laser_1.mat","C1701_V2Site2_3549Ch22_Laser_1.mat"=>"C1701_V2Site2_3549Ch22_Laser_1.mat"), Dict("C1701_V2Site2_3000Ch21_Laser_1.mat"=>"C1701_V2Site2_3000Ch21_Laser_1.mat","C1701_V2Site2_3549Ch22_Laser_1.mat"=>"C1701_V2Site2_3549Ch22_Laser_1.mat","C1701_V2Site2_3000Ch18_Laser_1.mat"=>"C1701_V2Site2_3000Ch18_Laser_1.mat","C1701_V2Site2_3000Ch22_Laser_1.mat"=>"C1701_V2Site2_3000Ch22_Laser_1.mat")), Any[], Any[], true, "horizontal", true)

# Prepare Dataset

In [148]:
dataset = prepare("$droot/$(sdir.value)/$(fname.value.value)","spike","digital")

Dict{Any,Any} with 7 entries:
  "digital"       => Dict{String,Any}(Pair{String,Any}("channel", Any[0x0005, 0…
  "spike"         => Dict{String,Any}(Pair{String,Any}("unitid", Any[UInt8[0x00…
  "source"        => "X:\\LZR1801\\V1b_2830\\LZR1801_V1b_2830_Laser_3"
  "ex"            => Dict{String,Any}(Pair{String,Any}("Subject_ID", "LZR1801")…
  "secondperunit" => 0.001
  "filepath"      => "Y:\\LZR1801\\LZR1801_V1b_2830_Laser_3.mat"
  "sourceformat"  => "Ripple"

## Condition Test

In [149]:
using Plots
ex = dataset["ex"];envparam = ex["EnvParam"];preicidur = ex["PreICI"];conddur = ex["CondDur"];suficidur = ex["SufICI"]
laser=filter!(i->!isempty(i),[ex["Param"]["Laser"],ex["Param"]["Laser2"]])[1]
laserbit = laser=="luxx473"?9:10
lasercolor = laser=="luxx473"?RGBA(0,0.7,1,1):RGBA(1,0.8,0,1)
ct,ctc=ctctc(ex)
vcti = .!isnan.(ct[:CondOnTime])
ct=ct[vcti,:];ctc=ctc[vcti,:]
condon = ct[:CondOn];condoff = ct[:CondOff];mconddur=mean(condoff-condon)
plotlyjs();default(size=(650,420), leg=true)
histogram(condoff-condon,bins=20,title="Condition Duration(Set to $conddur ms)")

## Use Laser Timing

In [150]:
bt,bv=digitalbit(digitaldata(dataset,Bits16DCh)...,laserbit)
dt,dv = digitaldata(dataset,CondDCh)
don=dt[dv.>0];doff=dt[dv.==0]
bts,ns,ws,idxs = subrv(bt,don,[don[2:end];Inf]);

In [151]:
len=100
plot(bt[1:len],bv[1:len],line=:steppost,linewidth=0,fillcolor=lasercolor,fillrange=1,label="LaserOn",xlabel="Time (ms)")
ceon=don[don.<bt[len]];ceoff=doff[doff.<bt[len]]
plot!(ceon,ones(length(ceon))*1.1,line=:sticks,color=:red,label="On")
plot!(ceoff,ones(length(ceoff))*1.1,line=:sticks,color=:blue,label="Off")

In [152]:
for i in 1:length(bts)
    if !isempty(bts[i])
        condon[i]=bts[i][1]
        condoff[i]=bts[i][end]
        bts[i]=bts[i]-condon[i]
    end
end
bvs=[bv[i] for i in idxs];

# Spiking Activity

In [153]:
spike = dataset["spike"];spikeeid = spike["electrodeid"];spikeuuid = spike["uuid"];spikeuid = spike["unitid"];spiketime = spike["time"]
eid = togglebuttons(spikeeid,label="Electrode")

Interact.Options{:ToggleButtons,Any}(97: "input-44" = 1 Any , "Electrode", 0x0001, "1", 1, Interact.OptionDict(DataStructures.OrderedDict{String,Any}("1"=>0x0001,"2"=>0x0002,"3"=>0x0003,"4"=>0x0004,"5"=>0x0005,"6"=>0x0006,"7"=>0x0007,"8"=>0x0008), Dict{Any,String}(Pair{Any,String}(0x0007, "7"),Pair{Any,String}(0x0004, "4"),Pair{Any,String}(0x0002, "2"),Pair{Any,String}(0x0003, "3"),Pair{Any,String}(0x0005, "5"),Pair{Any,String}(0x0008, "8"),Pair{Any,String}(0x0006, "6"),Pair{Any,String}(0x0001, "1"))), Any[], Any[], true, "horizontal", true)

In [154]:
eididx = findfirst(spikeeid.==eid.value);est = spiketime[eididx];esu = spikeuid[eididx];euuid = spikeuuid[eididx]
sts,ns,ws,is = subrv(est,condon-preicidur,condoff+suficidur,isminzero=true)
sts = map(x->x-preicidur,sts)
plotspiketrain(sts,uids=map(i->esu[i],is),timeline=[0,mconddur],title="Electrode$(eid.value)")

## Response

In [155]:
delay = slider(0:100,value=0,label="Delay")

Interact.Slider{Int64}(99: "input-45" = 0 Int64 , "Delay", 0, 0:100, "horizontal", true, "d", true)

In [156]:
urs = Dict(u=>subrvr(est[esu.==u],condon+delay.value,condoff+delay.value) for u in euuid);

In [157]:
factor = togglebuttons(finalfactor(ctc),label="Factor")

Interact.Options{:ToggleButtons,Symbol}(101: "input-46" = LaserFreq2 Symbol , "Factor", :LaserFreq2, "LaserFreq2", 1, Interact.OptionDict(DataStructures.OrderedDict("LaserFreq2"=>:LaserFreq2,"LaserPower2"=>:LaserPower2), Dict(:LaserFreq2=>"LaserFreq2",:LaserPower2=>"LaserPower2")), Any[], Any[], true, "horizontal", true)

In [241]:
plotcondresponse(urs,ctc,factor.value,title="Electrode$(eid.value)")

## PSTH

In [159]:
uid = map(e->togglebuttons(["All";spikeuuid[findfirst(spikeeid.==e)]],label="Unit"),signal(eid))

Interact.Options{:ToggleButtons,Any}(103: "input-47" = All Any , "Unit", "All", "All", 1, Interact.OptionDict(DataStructures.OrderedDict{String,Any}("All"=>"All","0"=>0x00,"1"=>0x01), Dict{Any,String}(Pair{Any,String}(0x00, "0"),Pair{Any,String}("All", "All"),Pair{Any,String}(0x01, "1"))), Any[], Any[], true, "horizontal", true)

In [160]:
usts,ns,ws,is = subrv(uid.value.value=="All"?est:est[esu.==uid.value.value],condon-preicidur,condoff+suficidur,isminzero=true)
usts = map(x->x-preicidur,usts);

In [161]:
bw = slider(2:20,value=10,label="BinWidth")

Interact.Slider{Int64}(107: "input-48" = 10 Int64 , "BinWidth", 10, 2:20, "horizontal", true, "d", true)

In [164]:
bin = -preicidur:bw.value:conddur+suficidur
plotpsth(usts,bin,ctc,factor.value,timeline=[0,mconddur],title="Electrode$(eid.value)_Unit$(uid.value.value)")

## Unique Condition Response

In [219]:
cond = condin(ctc)

,LaserFreq2,LaserPower2,n,i
1,"[0.0, 5.0, 0.0, 0.0]",1.0,100,"[1, 4, 8, 11, 13, 16, 20, 23, 25, 30, 31, 34, 38, 40, 45, 46, 51, 53, 56, 59, 62, 64, 68, 70, 75, 77, 81, 84, 87, 88, 93, 96, 98, 102, 105, 108, 111, 112, 117, 120, 123, 124, 128, 132, 133, 136, 140, 143, 145, 148, 151, 156, 157, 160, 163, 167, 171, 173, 175, 179, 181, 184, 189, 191, 195, 197, 200, 204, 206, 209, 211, 214, 217, 220, 224, 226, 229, 233, 236, 239, 241, 245, 249, 250, 253, 257, 260, 264, 266, 268, 272, 276, 278, 280, 285, 286, 290, 292, 296, 300]"
2,"[0.0, 0.0, 0.0, 0.0]",0.0,100,"[2, 5, 9, 12, 15, 17, 19, 24, 26, 29, 33, 35, 37, 42, 43, 47, 50, 52, 57, 60, 63, 66, 67, 71, 74, 76, 80, 83, 85, 89, 91, 94, 97, 101, 104, 106, 110, 114, 115, 119, 121, 126, 129, 130, 134, 137, 139, 144, 146, 150, 152, 155, 159, 162, 165, 168, 169, 172, 176, 178, 182, 186, 188, 190, 194, 196, 199, 203, 207, 208, 212, 216, 219, 221, 225, 228, 230, 232, 235, 238, 242, 244, 247, 252, 254, 258, 261, 263, 267, 269, 271, 275, 277, 281, 284, 287, 289, 293, 297, 298]"
3,"[0.0, 10.0, 0.0, 0.0]",1.0,100,"[3, 6, 7, 10, 14, 18, 21, 22, 27, 28, 32, 36, 39, 41, 44, 48, 49, 54, 55, 58, 61, 65, 69, 72, 73, 78, 79, 82, 86, 90, 92, 95, 99, 100, 103, 107, 109, 113, 116, 118, 122, 125, 127, 131, 135, 138, 141, 142, 147, 149, 153, 154, 158, 161, 164, 166, 170, 174, 177, 180, 183, 185, 187, 192, 193, 198, 201, 202, 205, 210, 213, 215, 218, 222, 223, 227, 231, 234, 237, 240, 243, 246, 248, 251, 255, 256, 259, 262, 265, 270, 273, 274, 279, 282, 283, 288, 291, 294, 295, 299]"


In [220]:
cstring = condstring(cond,finalfactor(cond))

3-element Array{String,1}:
 "LaserFreq2=[0.0, 5.0, 0.0, 0.0], LaserPower2=1.0" 
 "LaserFreq2=[0.0, 0.0, 0.0, 0.0], LaserPower2=0.0" 
 "LaserFreq2=[0.0, 10.0, 0.0, 0.0], LaserPower2=1.0"

In [212]:
ci=[2,3]
plotpsth(usts,bin,cond[ci,:],timeline=[0,mconddur],title="Electrode$(eid.value)_Unit$(uid.value.value)")

In [213]:
plot!(bts[3],bvs[3]*50,line=:steppost,linewidth=0,fillcolor=coloralpha(lasercolor,0.3),fillrange=1,label="LaserOn")

In [172]:
plotname=joinpath(rroot,"$(splitext(fname.value.value)[1])_E$(eid.value)_U$(uid.value.value)_10Hz_psth")
Plots.savefig("$plotname.png")
Plots.savefig("$plotname.svg")

In [221]:
ci=2
custs=usts[cond[ci,:i]];cuids=map(i->fill(uid.value.value,length(i)),is[cond[ci,:i]])
plotspiketrain(custs,uids=cuids,timeline=[0,mconddur],title="Electrode$(eid.value)_$(cstring[ci])")
plot!(bts[2],bvs[2],line=:steppost,linewidth=0,fillcolor=coloralpha(lasercolor,1),fillrange=1,label="LaserOn")

In [222]:
ci=1
custs=usts[cond[ci,:i]];cuids=map(i->fill(uid.value.value,length(i)),is[cond[ci,:i]])
plotspiketrain(custs,uids=cuids,timeline=[0,mconddur],title="Electrode$(eid.value)_$(cstring[ci])")
plot!(bts[1],bvs[1],line=:steppost,linewidth=0,fillcolor=coloralpha(lasercolor,1),fillrange=1,label="LaserOn")

In [223]:
ci=3
custs=usts[cond[ci,:i]];cuids=map(i->fill(uid.value.value,length(i)),is[cond[ci,:i]])
plotspiketrain(custs,uids=cuids,timeline=[0,mconddur],title="Electrode$(eid.value)_$(cstring[ci])")
plot!(bts[3],bvs[3],line=:steppost,linewidth=0,fillcolor=coloralpha(lasercolor,1),fillrange=1,label="LaserOn")

In [242]:
plotcondresponse(urs,cond,title="Electrode$(eid.value)")